# PETROV CLASSIFICATION OF SPACE-TIME IN ENTANGLED RELATIVITY FOR BOTH ELECTRICALLY AND MAGNETICALLY BLACK HOLES WHITHOUT ROTATION

This notebook's main objective is to characterize spacetime in the case of a non rotating black hole within the framework of entangled relativity (ER). Here, we use the Petrov classification criteria. As external solutions of isolated bodies characterized by their mass and angular momentum, we expect to find a type D black hole. 

We will start by introducing the ER metric, then we will calculate the Weyl tensor at first order in 'a' (the black hole's angular momentum), and finally, we will determine the NP-Weyl scalars to define the type of spacetime.

The complexity of some calculations in this notebook prompted us to perform intermediate first order Taylor expansions to maintain tractability of the different operations using the Sagemath language. This does not affect the validity of our results since they are part of a first-order analysis in 'a'.

In [1]:
version()

'SageMath version 10.1, Release Date: 2023-08-20'

In [2]:
%display latex

'SageMath version used is 10.1, Release Date: 2023-08-20'

In [3]:
from sage.manifolds.operators import dalembertian
from sage.manifolds.operators import grad
from sage.tensor.modules.tensor_with_indices import TensorWithIndices
import time
import pickle
comput_time0 = time.perf_counter()

Let's write a function to replace all the different expressions and perform the taylor expansions 

In [4]:
def slow_rot(arg):
    subs_funcs = [(U, A), (V, B), (X, C), (Y, D)]
    
    if hasattr(arg, 'expr'):
        arg = arg.expr()
        
    if hasattr(arg, 'apply_map')*hasattr(arg, 'display'):
        for old_func, new_func in subs_funcs:
            arg.apply_map(lambda f: f.substitute_function(old_func, new_func))
        arg.apply_map(lambda f: f.taylor(a, 0, 1).factor())
    else:
        for old_func, new_func in subs_funcs:
            arg = arg.substitute_function(old_func, new_func)
        arg = arg.taylor(a, 0, 1).factor() 
        
    return arg

# I. Definition of the metric in ER

In [5]:
M = Manifold(4, 'M', structure='Lorentzian')
print(M)

4-dimensional Lorentzian manifold M


In [6]:
X.<t,r,th,ph> = M.chart(r't r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi')
X

Chart (M, (t, r, th, ph))

In [7]:
g = M.metric()
r_m, r_p, a, Q = var('r_m r_p a Q')

In [8]:
U = function('U')
V = function('V')
X = function('X')
Y = function('Y')
g[0,0] = U(r)
g[1,1] = V(r)
g[2,2] = X(r)
g[3,3] = Y(r,th)

In [9]:
Psi(r) = (1-r_p/r)
Zeta(r) = (1-r_m/r)

A(r) = -Psi(r)*Zeta(r)**(7/13)
B(r) = 1/(Psi(r)*Zeta(r)**(15/13))
C(r) = r**2*Zeta(r)**(-2/13)
D(r) = (r*sin(th))**2*Zeta(r)**(-2/13)

Let's now define the mass and the charge expressions

In [10]:
Ma = (11/26)*r_m+r_p/2 
H=(r_m*r_p/(1+(1/(2*(3)**(1/2)))**2))**(1/2)

In [11]:
h = g.copy()
h.set_name('h')
slow_rot(h).display()

h = -(r - r_p)*((r - r_m)/r)^(7/13)/r dt⊗dt + r/((r - r_p)*((r - r_m)/r)^(15/13)) dr⊗dr + r^2/((r - r_m)/r)^(2/13) dth⊗dth + r^2*sin(th)^2/((r - r_m)/r)^(2/13) dph⊗dph

# II. Computation of the Weyl tensor

\begin{eqnarray}
C_{abcd} &=& R_{abcd} - \frac{1}{2}(g_{a[c}R_{d]b} - g_{b[c}R_{d]a}) + \frac{1}{6}R g_{a[c}g_{d]b}\\
&=& R_{abcd} - \frac{1}{2}\left(R_{ac}.g_{bd}-R_{ad}.g_{bc}+R_{bd}.g_{ac}-R_{bc}.g_{ad}\right)+\frac{1}{6}\left(g_{ac}g_{bd}-g_{ad}g_{bc}\right)R
\end{eqnarray}

We will not use the .weyl() property of SageMath here due to the complexity of the calculations and the need to progressively simplify the expressions through a Taylor expansion to first order in 'a'.

In [12]:
g.display()

g = U(r) dt⊗dt + V(r) dr⊗dr + X(r) dth⊗dth + Y(r, th) dph⊗dph

## II.1. The Riemann and Ricci tensors

In [13]:
ER_riem = g.riemann()
slow_rot(ER_riem)
ER_riem = ER_riem.down(g,0)
slow_rot(ER_riem)
ER_riem = TensorWithIndices(ER_riem, '_abcd')

In [14]:
ER_ricc = g.ricci()
slow_rot(ER_ricc)

Field of symmetric bilinear forms Ric(g) on the 4-dimensional Lorentzian manifold M

In [15]:
gu = g.up(g)
slow_rot(gu)

Tensor field of type (2,0) on the 4-dimensional Lorentzian manifold M

In [16]:
ER_rsc = gu['^ab']*ER_ricc['_ab']
show(LatexExpr(r'R = '), slow_rot(ER_rsc), LatexExpr(r'+ \mathcal{O}(a^{2})'))

R =  12/13*(r - r_m)^(2/13)*r_m*r_p/r^(54/13) + \mathcal{O}(a^{2})

## II.2. Antisymmetrization

1. Let's compute $Rg_{a[c}g_{d]b}$

In [17]:
g_prod = ER_rsc*g*g
slow_rot(g_prod)
gp_anti = g_prod['_acbd']-g_prod['_adbc']

antisymmetrize$\lbrace1,2\rbrace(g_{ac}g_{bd})\rightarrow \frac{1}{2}\left(g_{ac}g_{bd}-g_{ad}g_{bc}\right)=\frac{1}{2}g_{a[c}g_{b]d}$

2. Let's now compute $g_{a[c}R_{d]b} - g_{b[c}R_{d]a}$

In [18]:
gR = g*ER_ricc
slow_rot(gR)
gR_prod = gR['_acbd']-gR['_adbc']+gR['_bdac']-gR['_bcad']

# III. The Weyl tensor

In [19]:
ER_Weyl = ER_riem-(1/2)*gR_prod+(1/6)*gp_anti

for i in range(4):
    for j in range(4):
        for k in range(4):
            for l in range(4):
                latex_str = r'C_{{{}{}{}{}}} = '.format(i, j, k, l)  
                if [i, j, k, l] in [[0, 1, 0, 1],[0, 1, 1, 0],[0, 1, 1, 3],[0, 1, 2, 3],[0, 1, 3, 2]]:
                    show(LatexExpr(latex_str), ER_Weyl[i, j, k, l].expr().canonicalize_radical().factor().factor()) 

C_{0101} =  -1/169*(143*r^2*r_m - 140*r*r_m^2 + 169*r^2*r_p - 468*r*r_m*r_p + 296*r_m^2*r_p)/((r - r_m)^(19/13)*r^(46/13))

C_{0110} =  1/169*(143*r^2*r_m - 140*r*r_m^2 + 169*r^2*r_p - 468*r*r_m*r_p + 296*r_m^2*r_p)/((r - r_m)^(19/13)*r^(46/13))

C_{0113} =  0

C_{0123} =  0

C_{0132} =  0

# IV. NP-Weyl scalars

Let's import the tetrad expression in ER derived in the notebook entitled "ER_SR_BH_Tetrad"

In [31]:
elec = True

In [32]:
with open('NR_electric_tetrad_.pkl' if elec else 'NR_magnetic_tetrad_.pkl', 'rb') as file:
    expressions = pickle.load(file)
    
L = expressions['L_expression']
N = expressions['N_expression']
Mr = expressions['M_expression']
Mbar = expressions['Mbar_expression']

show(LatexExpr(r'l^{\mu} = '),L[:])
show(LatexExpr(r'n^{\mu} = '),N[:])
show(LatexExpr(r'm^{\mu} = '),Mr[:])
show(LatexExpr(r'\bar{m}^{\mu} = '), Mbar[:])

l^{\mu} =  [1/2*sqrt(2)*sqrt(-(-r_m/r + 1)^(11/13)*(r_p/r - 1))/((-r_m/r + 1)^(11/13)*(r_p/r - 1)),
 1/2*sqrt(2)/sqrt(1/((-r_m/r + 1)^(11/13)*(r_p/r - 1))),
 0,
 0]

n^{\mu} =  [1/2*sqrt(2)*sqrt(-(-r_m/r + 1)^(11/13)*(r_p/r - 1))/((-r_m/r + 1)^(11/13)*(r_p/r - 1)),
 -1/2*sqrt(2)/sqrt(1/((-r_m/r + 1)^(11/13)*(r_p/r - 1))),
 0,
 0]

m^{\mu} =  [0,
 0,
 1/2*sqrt(2)/(r*(-r_m/r + 1)^(1/13)),
 1/2*I*sqrt(2)/(r*(-r_m/r + 1)^(1/13)*abs(sin(th)))]

\bar{m}^{\mu} =  [0,
 0,
 1/2*sqrt(2)/(r*(-r_m/r + 1)^(1/13)),
 -1/2*I*sqrt(2)/(r*(-r_m/r + 1)^(1/13)*abs(sin(th)))]

In [33]:
tetrad_L, tetrad_N, tetrad_Mr,tetrad_Mbar = M.vector_field(name='tetrad_L'), M.vector_field(name='tetrad_N'), \
                                            M.vector_field(name='tetrad_Mr'), M.vector_field(name='tetrad_Mbar')
for i in range(4):
    tetrad_L[i] = L[i].expr()
    tetrad_N[i] = N[i].expr()
    tetrad_Mr[i] = Mr[i].expr()
    tetrad_Mbar[i] = Mbar[i].expr()

In [34]:
components = [[[[ER_Weyl[i,j,k,l] for i in range(4)] for j in range(4)] for k in range(4)] for l in range(4)]
Weyl_tensor = M.tensor_field(0, 4, name='Weyl_tensor')
Weyl_tensor[:, :, :, :] = components

## IV.1. Scalar 0

In [35]:
print('1st contraction', Weyl_tensor.tensor_type())
Psi0 = Weyl_tensor.contract(0, tetrad_L, 0)
Psi0.apply_map(lambda u: u.taylor(a,0,1))
print('2nd contraction', Psi0.tensor_type())
Psi0 = Psi0.contract(0, tetrad_Mr, 0)
Psi0.apply_map(lambda u: u.taylor(a,0,1))
print('3rd contraction', Psi0.tensor_type())
Psi0 = Psi0.contract(0, tetrad_L, 0)
Psi0.apply_map(lambda u: u.taylor(a,0,1))
print('4th contraction', Psi0.tensor_type())
Psi0 = Psi0.contract(0, tetrad_Mr, 0)
Psi0 = Psi0.expr().taylor(a,0,1)
latex_str = r'\Psi_{0} = W_{\alpha\beta\gamma\delta}l^{\alpha}m^{\beta}l^{\gamma}m^{\delta} = ' 
show(LatexExpr(latex_str), Psi0)

1st contraction (0, 4)
2nd contraction (0, 3)
3rd contraction (0, 2)
4th contraction (0, 1)


\Psi_{0} = W_{\alpha\beta\gamma\delta}l^{\alpha}m^{\beta}l^{\gamma}m^{\delta} =  0

## IV.2. Scalar I

In [36]:
print('1st contraction', Weyl_tensor.tensor_type())
Psi1 = Weyl_tensor.contract(0, tetrad_L, 0)
Psi1.apply_map(lambda u: u.taylor(a,0,1))
print('2nd contraction', Psi1.tensor_type())
Psi1 = Psi1.contract(0, tetrad_N, 0)
Psi1.apply_map(lambda u: u.taylor(a,0,1))
print('3rd contraction', Psi1.tensor_type())
Psi1 = Psi1.contract(0, tetrad_L, 0)
Psi1.apply_map(lambda u: u.taylor(a,0,1))
print('4th contraction', Psi1.tensor_type())
Psi1 = Psi1.contract(0, tetrad_Mr, 0)
Psi1 = Psi1.expr().taylor(a,0,1)
latex_str = r'\Psi_{1} = W_{\alpha\beta\gamma\delta}l^{\alpha}n^{\beta}l^{\gamma}m^{\delta} = ' 
show(LatexExpr(latex_str), Psi1)

1st contraction (0, 4)
2nd contraction (0, 3)
3rd contraction (0, 2)
4th contraction (0, 1)


\Psi_{1} = W_{\alpha\beta\gamma\delta}l^{\alpha}n^{\beta}l^{\gamma}m^{\delta} =  0

## IV.3. Scalar II

In [37]:
print('1st contraction', Weyl_tensor.tensor_type())
Psi2 = Weyl_tensor.contract(0, tetrad_L, 0)
Psi2.apply_map(lambda u: u.taylor(a,0,1))
print('2nd contraction', Psi2.tensor_type())
Psi2 = Psi2.contract(0, tetrad_Mr, 0)
Psi2.apply_map(lambda u: u.taylor(a,0,1))
print('3rd contraction', Psi2.tensor_type())
Psi2 = Psi2.contract(0, tetrad_Mbar, 0)
Psi2.apply_map(lambda u: u.taylor(a,0,1))
print('4th contraction', Psi2.tensor_type())
Psi2 = Psi2.contract(0, tetrad_N, 0)
Psi2 = Psi2.expr().taylor(a,0,1)
latex_str = r'\Psi_{2} = W_{\alpha\beta\gamma\delta}l^{\alpha}m^{\beta}\bar{m}^{\gamma}n^{\delta} = ' 
show(LatexExpr(latex_str), Psi2)

1st contraction (0, 4)
2nd contraction (0, 3)
3rd contraction (0, 2)
4th contraction (0, 1)


\Psi_{2} = W_{\alpha\beta\gamma\delta}l^{\alpha}m^{\beta}\bar{m}^{\gamma}n^{\delta} =  -1/676*(143*r^3*r_m - 283*r^2*r_m^2 + 140*r*r_m^3 + (169*r^3 - 637*r^2*r_m + 764*r*r_m^2 - 296*r_m^3 + (169*(r - r_m)^(2/13)*r^2 - 468*(r - r_m)^(2/13)*r*r_m + 296*(r - r_m)^(2/13)*r_m^2)*abs(-r + r_m)^(11/13))*r_p + (143*(r - r_m)^(2/13)*r^2*r_m - 140*(r - r_m)^(2/13)*r*r_m^2)*abs(-r + r_m)^(11/13))/((r - r_m)^(6/13)*r^(72/13) - 2*(r - r_m)^(6/13)*r^(59/13)*r_m + (r - r_m)^(6/13)*r^(46/13)*r_m^2)

## IV.4. Scalar III

In [38]:
print('1st contraction', Weyl_tensor.tensor_type())
Psi3 = Weyl_tensor.contract(0, tetrad_L, 0)
Psi3.apply_map(lambda u: u.taylor(a,0,1))
print('2nd contraction', Psi3.tensor_type())
Psi3 = Psi3.contract(0, tetrad_N, 0)
Psi3.apply_map(lambda u: u.taylor(a,0,1))
print('3rd contraction', Psi3.tensor_type())
Psi3 = Psi3.contract(0, tetrad_Mbar, 0)
Psi3.apply_map(lambda u: u.taylor(a,0,1))
print('4th contraction', Psi3.tensor_type())
Psi3 = Psi3.contract(0, tetrad_N, 0)
Psi3 = Psi3.expr().taylor(a,0,1)
latex_str = r'\Psi_{3} = W_{\alpha\beta\gamma\delta}l^{\alpha}n^{\beta}\bar{m}^{\gamma}n^{\delta} = ' 
show(LatexExpr(latex_str), Psi3)

1st contraction (0, 4)
2nd contraction (0, 3)
3rd contraction (0, 2)
4th contraction (0, 1)


\Psi_{3} = W_{\alpha\beta\gamma\delta}l^{\alpha}n^{\beta}\bar{m}^{\gamma}n^{\delta} =  0

## IV.5. Scalar IV

In [39]:
print('1st contraction', Weyl_tensor.tensor_type())
Psi4 = Weyl_tensor.contract(0, tetrad_N, 0)
Psi4.apply_map(lambda u: u.taylor(a,0,1))
print('2nd contraction', Psi4.tensor_type())
Psi4 = Psi4.contract(0, tetrad_Mbar, 0)
Psi4.apply_map(lambda u: u.taylor(a,0,1))
print('3rd contraction', Psi4.tensor_type())
Psi4 = Psi4.contract(0, tetrad_N, 0)
Psi4.apply_map(lambda u: u.taylor(a,0,1))
print('4th contraction', Psi4.tensor_type())
Psi4 = Psi4.contract(0, tetrad_Mbar, 0)
Psi4 = Psi4.expr().taylor(a,0,1)
latex_str = r'\Psi_{4} = W_{\alpha\beta\gamma\delta}n^{\alpha}\bar{m}^{\beta}n^{\gamma}\bar{m}^{\delta} = ' 
show(LatexExpr(latex_str), Psi4)

1st contraction (0, 4)
2nd contraction (0, 3)
3rd contraction (0, 2)
4th contraction (0, 1)


\Psi_{4} = W_{\alpha\beta\gamma\delta}n^{\alpha}\bar{m}^{\beta}n^{\gamma}\bar{m}^{\delta} =  0

# V. Properties of $\Psi_{2}$

## V.I. Near vacuum limit

In the case of small $r_{-}$ for an external solution $r>r_{p}$ we can write:

In [40]:
assume(r > r_p)
psi_taylor = Psi2.taylor(r_m,0,1)
show(LatexExpr(r'\Psi_{2} = '),psi_taylor,LatexExpr(r' + \mathfrak{O}(r_m^2)'))

\Psi_{2} =  -1/26*(11*r - 17*r_p)*r_m/r^4 - 1/2*r_p/r^3  + \mathfrak{O}(r_m^2)

Let's check that in the case ($r_{m}=0$), the NP-Weyl scalar $\Psi_{2}$ has the same expression that in the case of the Schwarzchild solution.
$\Psi_{2}^{schwarz}=-\frac{M}{r^{3}}$


Let's start by replacing $M$ by its expressions, and show that $\lim_{r_m \to 0}\Psi_{2} = \Psi_{2}^{schwarz}$

In [41]:
show(LatexExpr('M = '), Ma)
show(LatexExpr(r'\Psi_{2}^{schwarz} = '), -(Ma/(r**3)).subs(r_m=0))

M =  11/26*r_m + 1/2*r_p

\Psi_{2}^{schwarz} =  -1/2*r_p/r^3

Finally, we verify

In [42]:
assume(r > r_p)
show(LatexExpr(r'\lim_{r_m \to 0}\Psi_{2} = '),psi_taylor.subs(r_m=0),LatexExpr(r' = \Psi_{2}^{schwarz}'))

\lim_{r_m \to 0}\Psi_{2} =  -1/2*r_p/r^3  = \Psi_{2}^{schwarz}

So in vacuum, we recover the Schwarzchild solution.

# In conclusion, and as expected, we find a type D solution for both the electric and magnetic cases.